### ours

In [ ]:
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, precision_recall_curve, f1_score
from sklearn.model_selection import RepeatedKFold
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import ssl
from joblib import dump, load

from sklearn.feature_selection import SelectFromModel
from itertools import combinations
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context
print(np.__file__)
print(np.__version__) 
subtype = 'PosNeg'
neworold = 'new'
if subtype=='NegNeg':
    clin_feature = ['T','N','Ki-67','G3']
else:
    clin_feature = ['T','N','G3']
clin_feature
df_train=pd.read_csv("./table.4-2.ER-HER2+.Top3_for_Each_Omics.Tier1.txt",sep="\t",index_col=0)
df_train = df_train[df_train.columns[~df_train.columns.str.contains('meth_') | (df_train.columns == 'meth_KIT@chr4_54657389_54657742')]]


# print(df_train['variable'])

kf = RepeatedKFold(n_splits=5, n_repeats=30, random_state=1)

fs = {}
fs['clin'] = clin_feature
fs['omics'] = clin_feature+list(df_train.columns[df_train.columns.str.contains('meth_|rna_|pro_|phospro_')])

if len(df_train.columns[df_train.columns.str.contains('meth_')])>0:
    fs['single.meth'] = list(df_train.columns[df_train.columns.str.contains('meth_')])

if len(df_train.columns[df_train.columns.str.contains('rna_')])>0:
    fs['single.rna'] = list(df_train.columns[df_train.columns.str.contains('rna_')])
    
if len(df_train.columns[df_train.columns.str.contains('pro_')])>0:
    fs['single.prot'] = list(df_train.columns[(df_train.columns.str.contains('pro_')) & (~df_train.columns.str.contains('phospro_'))])

if len(df_train.columns[df_train.columns.str.contains('phospro_')])>0:
    fs['single.phospro'] = list(df_train.columns[df_train.columns.str.contains('phospro_')])

# 初始化统计信息
feature_selection_stats = {
    'activated_count': 0,
    'selected_features': {},
    'avg_features_count': []
}

aucs = {}
aps = {}
accuracys = {}
for n_est in [50,100,200]:
    for l_rate in [0.05,0.1,0.2]:
        aucs[(n_est, l_rate)] = {}
        aps[(n_est, l_rate)] = {}
        accuracys[(n_est, l_rate)] = {}
        for f in fs.keys():
            aucs[(n_est, l_rate)][f] = []
            aps[(n_est, l_rate)][f] = []
            accuracys[(n_est, l_rate)][f] = []
    
for n_est in [50,100,200]:
    for l_rate in [0.05,0.1,0.2]:
        print(f"Training model with {n_est} trees and {l_rate} learning rate")
        for train_index, test_index in kf.split(df_train):
            X_train, X_test = df_train.drop(columns='pCR').iloc[list(train_index)], df_train.drop(columns='pCR').iloc[list(test_index)]
            y_train, y_test = df_train['pCR'].iloc[list(train_index)], df_train['pCR'].iloc[list(test_index)]
            
            if y_train.unique().shape[0]==2 and y_test.unique().shape[0]==2:
                # 先训练单一组学模型，获取性能基准
                single_omics_performance = {}
                single_omics_keys = [k for k in fs.keys() if k.startswith('single')]
                
                for single_key in single_omics_keys:
                    single_model = lgb.LGBMClassifier(verbose=-1, random_state=1, n_estimators=n_est, 
                                                    learning_rate=l_rate, min_child_samples=2,
                                                    reg_alpha=0.2, reg_lambda=0.2, max_depth=5)
                    single_model.fit(X_train[fs[single_key]], y_train)
                    single_preds = [i[1] for i in single_model.predict_proba(X_test[fs[single_key]])]
                    single_auc = roc_auc_score(y_test, single_preds)
                    precision, recall, _ = precision_recall_curve(y_test, single_preds)
                    single_ap = auc(recall, precision)
                    single_omics_performance[single_key] = (single_auc, single_ap)
                    X_test[single_key] = single_preds
                    single_preds_binary = (np.array(single_preds) >= 0.5).astype(int)
                    aucs[(n_est, l_rate)][single_key].append(single_auc)
                    aps[(n_est, l_rate)][single_key].append(single_ap)
                    accuracys[(n_est, l_rate)][single_key].append(metrics.accuracy_score(y_test, single_preds_binary))
                
                # 训练临床特征模型
                clin_model = lgb.LGBMClassifier(verbose=-1, random_state=1, n_estimators=n_est, 
                                              learning_rate=l_rate, min_child_samples=2,
                                              reg_alpha=0.2, reg_lambda=0.2, max_depth=5)
                clin_model.fit(X_train[fs['clin']], y_train)
                clin_preds = [i[1] for i in clin_model.predict_proba(X_test[fs['clin']])]
                clin_auc = roc_auc_score(y_test, clin_preds)
                precision, recall, _ = precision_recall_curve(y_test, clin_preds)
                clin_ap = auc(recall, precision)
                X_test['clin'] = clin_preds
                clin_preds_binary = (np.array(clin_preds) >= 0.5).astype(int)
                aucs[(n_est, l_rate)]['clin'].append(clin_auc)
                aps[(n_est, l_rate)]['clin'].append(clin_ap)
                accuracys[(n_est, l_rate)]['clin'].append(metrics.accuracy_score(y_test, clin_preds_binary))
                best_single_auc = max([perf[0] for perf in single_omics_performance.values()])
                best_single_ap = max([perf[1] for perf in single_omics_performance.values()])
                base_features = clin_feature.copy()
                non_clin_features = [f for f in fs['omics'] if f not in clin_feature]
                feature_selection_stats['activated_count'] += 1
                
                best_feature_set = None
                best_auc_diff = float('inf')
                best_valid_auc = 0
                all_combinations = []
                for r in range(1, len(non_clin_features) + 1):
                    all_combinations.extend(list(combinations(non_clin_features, r)))
                
                # 评估每个特征组合
                for combo in all_combinations:
                    feature_set = base_features + list(combo)
                    
                    # 训练模型
                    model = lgb.LGBMClassifier(verbose=-1, random_state=1, n_estimators=n_est,
                                             learning_rate=l_rate, min_child_samples=2,
                                             reg_alpha=0.2, reg_lambda=0.2, max_depth=5)
                    model.fit(X_train[feature_set], y_train)
                    
                    # 在训练集上评估
                    train_preds = [i[1] for i in model.predict_proba(X_train[feature_set])]
                    train_auc = roc_auc_score(y_train, train_preds)
                    
                    # 在验证集上评估
                    valid_preds = [i[1] for i in model.predict_proba(X_test[feature_set])]
                    valid_auc = roc_auc_score(y_test, valid_preds)
                    
                    # 计算AUC差异
                    auc_diff = abs(train_auc - valid_auc)
                    if (auc_diff < best_auc_diff and valid_auc >= best_single_auc):
                        best_auc_diff = auc_diff
                        best_feature_set = feature_set
                        best_valid_auc = valid_auc
                if best_feature_set is None:
                    best_single_omics = max(single_omics_performance.items(), key=lambda x: x[1][0])[0]
                    best_feature_set = clin_feature + fs[best_single_omics]
                final_model = lgb.LGBMClassifier(verbose=-1, random_state=1, n_estimators=n_est,
                                                learning_rate=l_rate, min_child_samples=2,
                                                reg_alpha=0.2, reg_lambda=0.2, max_depth=5)
                final_model.fit(X_train[best_feature_set], y_train)
                omics_preds = [i[1] for i in final_model.predict_proba(X_test[best_feature_set])]
                omics_auc = roc_auc_score(y_test, omics_preds)
                precision, recall, _ = precision_recall_curve(y_test, omics_preds)
                omics_ap = auc(recall, precision)
                omics_preds_binary = (np.array(omics_preds) >= 0.5).astype(int)  # 二值化预测结果
                # 记录选择的特征
                feature_selection_stats['selected_features'][len(feature_selection_stats['selected_features'])] = best_feature_set
                feature_selection_stats['avg_features_count'].append(len(best_feature_set))
                
                # 记录最终的多组学模型性能
                X_test['omics'] = omics_preds
                aucs[(n_est, l_rate)]['omics'].append(omics_auc)
                aps[(n_est, l_rate)]['omics'].append(omics_ap)
                accuracys[(n_est, l_rate)]['omics'].append(metrics.accuracy_score(y_test, omics_preds_binary))
def best_params(res, fs):
    best_pdict = {}
    for i in fs:
        best_pdict[i] = pd.DataFrame(res).applymap(sum).T.sort_values(i).index[-1]
    return pd.concat([pd.DataFrame(res[best_pdict[i]]).melt().loc[pd.DataFrame(res[best_pdict[i]]).melt()['variable']==i] for i in fs])

for m in [aucs, aps]:
    for i in fs.keys():
        print('Best params for '+i+' model:')
        print('# of trees:',pd.DataFrame(m).applymap(sum).T[i].idxmax()[0], '\nLearning rate:',pd.DataFrame(m).applymap(sum).T[i].idxmax()[1])
        df = best_params(m, [i,])
        print(df[df['variable']==i]['value'].mean())
        print()
    print()

# 输出特征选择统计信息
if feature_selection_stats['activated_count'] > 0:
    
    # 显示最常被选择的特征
    if feature_selection_stats['selected_features']:
        feature_counts = {}
        for feature_list in feature_selection_stats['selected_features'].values():
            for feature in feature_list:
                if feature not in clin_feature: 
                    feature_counts[feature] = feature_counts.get(feature, 0) + 1
        
        for feature, count in sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"{feature}: {count} 次")

res_dict = {
    'aucs':aucs,
    'aps':aps,
    'accuracy':accuracys,
}

for resi in res_dict.keys():
    res = res_dict[resi]
    winner = best_params(res, fs.keys()).groupby('variable').mean().sort_values('value').index[-1]
    runnerup = best_params(res, fs.keys()).groupby('variable').mean().sort_values('value').index[-2]
    df = best_params(res, [runnerup,winner,])
    
    f, ax = plt.subplots(figsize=[1.35*5,3])
    df = best_params(res, fs.keys())
    sns.boxplot(x=df['variable'], y=df['value'], color='#ffffff', width=0.75, showmeans=1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.ylim([0.5,1.02])
    plt.ylabel(resi)

    print('winner vs runnerup:')
    print(st.ranksums(df[df['variable']==runnerup]['value'], df[df['variable']==winner]['value']))
    print((df[df['variable']==runnerup]['value'].mean(), df[df['variable']==winner]['value'].mean()))
df.to_csv('fig_6b_s7b_final_data_her2e.csv')   
# 获取最佳超参数
best_n_est = pd.DataFrame(accuracys).applymap(sum).T['omics'].idxmax()[0]
best_lr = pd.DataFrame(accuracys).applymap(sum).T['omics'].idxmax()[1]

# 确定最终使用的特征集
if feature_selection_stats['activated_count'] > 0:
    feature_counts = {}
    for feature_list in feature_selection_stats['selected_features'].values():
        for feature in feature_list:
            feature_counts[feature] = feature_counts.get(feature, 0) + 1
    
    selection_threshold = feature_selection_stats['activated_count'] * 0.3
    final_features = [f for f in clin_feature] 
    for feature, count in feature_counts.items():
        if count >= selection_threshold and feature not in final_features:
            final_features.append(feature)
    
else:
    final_features = fs['omics']

# 训练最终模型
model = lgb.LGBMClassifier(verbose=-1, random_state=1, 
                           n_estimators=best_n_est, 
                           learning_rate=best_lr,
                           min_child_samples=2,
                           reg_alpha=0.2,
                           reg_lambda=0.2,
                           max_depth=5)
model.fit(df_train[final_features], df_train['pCR'])
model_info = {
    'model': model,
    'feature_names': X_train.columns.tolist(),
    'training_params': model.get_params()
}
dump(model_info, 'her2e_newfeature_newdata_250314_lgb.joblib')
# 保存模型
model.booster_.save_model('./'+subtype+'-'+neworold+'.model')

# 加载模型用于SHAP分析
import shap
model_booster = lgb.Booster(model_file='./'+subtype+'-'+neworold+'.model')
model_booster.params['objective'] = 'binary'

# 创建SHAP解释器
explainer = shap.TreeExplainer(model_booster)

# 计算SHAP值
shap_values = explainer.shap_values(df_train[final_features])

# 绘制SHAP摘要图
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values[1], df_train[final_features], show=False, plot_type="dot")
plt.tight_layout()
plt.savefig('./shap_her2e_optimized.pdf', bbox_inches='tight', dpi=600)

# 额外绘制特征重要性条形图
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values[1], df_train[final_features], show=False, plot_type="bar")
plt.tight_layout()
# plt.savefig('./shap_tn_bar_optimized.pdf', bbox_inches='tight', dpi=300)

# 输出最终使用的特征及其类型统计
feature_types = {
    'clinical': [],
    'methylation': [],
    'rna': [],
    'protein': [],
    'phosphoprotein': []
}

for feature in final_features:
    if feature in clin_feature:
        feature_types['clinical'].append(feature)
    elif 'meth_' in feature:
        feature_types['methylation'].append(feature)
    elif 'rna_' in feature:
        feature_types['rna'].append(feature)
    elif 'phospro_' in feature:
        feature_types['phosphoprotein'].append(feature)
    elif 'pro_' in feature:
        feature_types['protein'].append(feature)

print("\n最终模型使用的特征类型统计:")
for ftype, features in feature_types.items():
    print(f"{ftype}: {len(features)}个特征")

#### test on transneo

In [ ]:
import pandas as pd
from scipy import stats as st
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, auc, f1_score
from sklearn.model_selection import RepeatedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
df_transneo = pd.read_csv('./final_data/table.1-1.IndependentVal_Sammut2021Dis.ER-HER2+.txt', index_col=0, sep='\t')
# df_transneo
df_supp=pd.read_csv('./transneo_clinomics_her2e.csv', index_col=0)

df_transneo['TransNEO'] = df_supp['TransNEO']
df_transneo_fake = df_transneo.copy()
df_transneo

f = 'Pr'
kit = pd.read_csv('./LASSO/independent_validation_kit_predictions.csv', index_col=0)
df_transneo['meth_KIT@chr4_54657389_54657742'] = kit.loc[df_transneo.index]['Predicted_kit']

df_transneo_fake['meth_KIT@chr4_54657389_54657742'] = list(st.zscore(pd.Series(-1*df_transneo_fake['meth_KIT@chr4_54657389_54657742'])))

df_transneo[f] = model.predict(df_transneo[model.feature_name_].astype(float))
proba = model.predict_proba(df_transneo[model.feature_name_].astype(float))
proba_fake = model.predict_proba(df_transneo_fake[model.feature_name_].astype(float))
# print(proba)
pd.DataFrame(proba, index=df_transneo.index).to_csv('her2e_transneo_prob.txt')

# df_transneo[f].to_csv('transneoDis_her2e_prediction.txt')
roc_auc_score(df_transneo['pCR'],proba[:,1]),\
roc_auc_score(df_transneo['pCR'],df_transneo['TransNEO'])

# ROC曲线
fig, ax = plt.subplots(figsize=[3,3])

# TransNEO的ROC曲线
fpr, tpr, threshold = metrics.roc_curve(df_transneo['pCR'],df_transneo['TransNEO'])
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label = 'TransNEO AUC = %0.2f' % roc_auc, linestyle='dotted', linewidth=3, color='darkgray')
print('TransNEO ROC:', roc_auc)

# 模型的ROC曲线
fpr, tpr, threshold = metrics.roc_curve(df_transneo['pCR'],proba[:,1])
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label = 'Model AUC = %0.2f' % roc_auc, linewidth=3, color='gray')
print('Model ROC:', roc_auc)

# Fake数据的ROC曲线
fpr_fake, tpr_fake, _ = metrics.roc_curve(df_transneo['pCR'], proba_fake[:,1])
roc_auc_fake = metrics.auc(fpr_fake, tpr_fake)
plt.plot(fpr_fake, tpr_fake, label='Normalized AUC = %.2f' % roc_auc_fake,
         linewidth=3, color='black')
print('Normalized ROC:', roc_auc_fake)
pd.DataFrame(proba_fake, index=df_transneo_fake.index).to_csv('her2e_transneo_prob_fake.txt')
pd.DataFrame(proba, index=df_transneo.index).to_csv('her2e_transneo_prob.txt')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.legend()
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.yticks([0,0.5,1])
plt.xticks([0,0.5,1])

# PR曲线
fig, ax = plt.subplots(figsize=[3,3])

# TransNEO的PR曲线
precision_transneo, recall_transneo, _ = precision_recall_curve(df_transneo['pCR'], df_transneo['TransNEO'])
auprc_transneo = auc(recall_transneo, precision_transneo)
plt.plot(recall_transneo, precision_transneo, label = 'TransNEO AP = %0.2f' % auprc_transneo, 
         linestyle='dotted', linewidth=3, color='darkgray')
print('TransNEO AP:', auprc_transneo)

# 模型的PR曲线
precision_model, recall_model, _ = precision_recall_curve(df_transneo['pCR'], proba[:,1])
auprc_model = auc(recall_model, precision_model)
plt.plot(recall_model, precision_model, label = 'Model AP = %0.2f' % auprc_model, 
         linewidth=3, color='gray')
print('Model AP:', auprc_model)

# Fake数据的PR曲线
precision_fake, recall_fake, _ = precision_recall_curve(df_transneo['pCR'], proba_fake[:,1])
auprc_fake = auc(recall_fake, precision_fake)
plt.plot(recall_fake, precision_fake, label = 'Normalized AP = %0.2f' % auprc_fake,
         linewidth=3, color='black')
print('Normalized AP:', auprc_fake)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.legend()
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.yticks([0,0.5,1])
plt.xticks([0,0.5,1])
plt.savefig('transneoDis_her2e_combined_ap.pdf',dpi=600)

#### test on sammut validation

In [ ]:
from scipy.stats import mannwhitneyu
df_sammutval = pd.read_csv('./final_data/table.2-1.IndependentVal_Sammut2021Val.ER-HER2+.txt', index_col=0, sep='\t')

kit = pd.read_csv('./LASSO/sam_independent_validation_kit_predictions.csv', index_col=0)
df_sammutval['meth_KIT@chr4_54657389_54657742'] = kit.loc[df_sammutval.index]['Predicted_kit']

df_sammutval['G3']=float('nan')
proba = model.predict_proba(df_sammutval[model.feature_name_].astype(float))
df_sammutval['Pr'] = proba[:,1]
df_sammutval['Pr'].to_csv('her2e_sammutval_prediction.txt')
df_sammutval

# ROC curve
plt.figure(figsize=(3,3))
ax = plt.gca()
fpr, tpr, threshold = metrics.roc_curve(df_sammutval['pCR'],df_sammutval['Pr'])
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label = 'AUC = %0.2f' % roc_auc, linewidth=3, color='gray')
print(roc_auc)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.yticks([0,0.5,1])
plt.xticks([0,0.5,1])

# Box plot
plt.figure(figsize=(3,3))
ax = plt.gca()
sns.boxplot(x='pCR', y='Pr', data=df_sammutval)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel('pCR')
plt.ylabel('Predicted probability')

# Calculate p-value using Mann-Whitney U test
pCR_scores = df_sammutval.loc[df_sammutval['pCR']==1, 'Pr']
RD_scores = df_sammutval.loc[df_sammutval['pCR']==0, 'Pr']
stat, pval = mannwhitneyu(pCR_scores, RD_scores, alternative='two-sided')
print(f'p-value: {pval:.2e}')